# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the citibike dataset - check that you are able to read in about 40k rows

In [1]:
!pip install --user urllib3

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --user future

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='
# CartoDB: http://nyu.carto.com/u/fb55/tables/citibike/public

import __future__

from __future__ import print_function
__author__ = "Federica B. Bianco, CUSP NYU 2018"

#try:
 #   import urllib2 as ulib
#except ImportError:
    #import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io
    
try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

import sys

if sys.version_info.major > 2:
    xrange = range
import six
    

In [4]:
# For Loop is not available for SQL. 

API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [5]:
query = '''SELECT * FROM fb55.citibike
WHERE tripduration > 1000
LIMIT 5
'''

table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
3,NaN,Broadway & W 36 St,NaN,20481,-73.987654,1001,2015-02-04 19:59:00+00,2015-02-04 20:15:00+00,267,20425,40.750977,504,1 Ave & E 15 St,40.732219,-73.981656,19935,Subscriber,1985.0,2
4,NaN,8 Ave & W 31 St,NaN,21285,-73.994811,1618,2015-02-04 21:58:00+00,2015-02-04 22:25:00+00,521,21228,40.750450,483,E 12 St & 3 Ave,40.732233,-73.988900,14660,Subscriber,1976.0,1


In [6]:
test = '''
SELECT *
FROM citibike_short
'''

In [7]:
a = pd.read_csv(io(queryCartoDB(test)), sep=',')
a.head(3)

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.994811,NaN,801,2/1/2015 0:00,2/1/2015 0:14,521,8 Ave & W 31 St,40.750450,1,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978.0,2
1,NaN,-73.985649,NaN,245,2/1/2015 1:03,2/1/2015 1:07,438,St Marks Pl & 1 Ave,40.727791,89,297,E 15 St & 3 Ave,40.734232,-73.986923,20142,Subscriber,1990.0,1
2,NaN,-73.990093,NaN,379,2/1/2015 0:00,2/1/2015 0:07,497,E 17 St & Broadway,40.737050,2,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993.0,1


## Task 1 — Familiarize with SQL Clauses

In [8]:
# Reference: https://www.postgresql.org/docs/9.0/static/functions-datetime.html
# Sort data by start_station_id, tripduration - Only checking trips with duration <= 3 hours
query = '''
SELECT start_station_id, tripduration/3600 as TripLength FROM fb55.citibike
WHERE tripduration/3600 <= 3
Group BY start_station_id, tripduration/3600
ORDER BY tripduration/3600 DESC
LIMIT 10'''
table = get_data(query)
table

,start_station_id,triplength
0,511,3
1,469,3
2,492,3
3,347,3
4,327,3
5,251,3
6,137,3
7,518,3
8,490,3
9,402,2


In [9]:
# List all unique start_station_id values 
query = '''
SELECT DISTINCT start_station_id FROM fb55.citibike'''

table = get_data(query)
table.head()

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


In [10]:
# • Aggregation functions:
# - Count the number of trips (aka wc -l in SQL)
query = '''SELECT COUNT(cartodb_id) FROM fb55.citibike'''
table = get_data(query)
table.head()


,count,Unnamed: 1
0,46200,NaN


In [11]:
# - Find the average/min/max trip duration
query = '''
SELECT 
MAX(tripduration) as Max,
MIN(tripduration) as Min,
AVG(tripduration) as Mean 
FROM fb55.citibike;
'''
table = get_data(query)
table.head()


,max,min,mean
0,43016,60,675.865823


## Task 2 — Working with date/time

In [12]:
# Selecting trips started on Feb-02-2015 only 
query = '''
SELECT starttime FROM fb55.citibike
WHERE EXTRACT(DAY FROM starttime::date)=02 '''
table = get_data(query)
table.head()

,starttime,Unnamed: 1
0,2015-02-02 17:44:00+00,NaN
1,2015-02-02 20:12:00+00,NaN
2,2015-02-02 11:07:00+00,NaN
3,2015-02-02 00:40:00+00,NaN
4,2015-02-02 00:40:00+00,NaN


In [13]:
# Selecting trips started on the weekends
query = '''
SELECT AVG(tripduration) as Trip_average_wkend FROM fb55.citibike
WHERE EXTRACT (DOW FROM starttime::date) IN (6,0)



'''
table = get_data(query)
table.head()

,trip_average_wkend,Unnamed: 1
0,662.942181,NaN


In [14]:
# Can we do the same for weekday?
query = '''
SELECT AVG(tripduration) as Trip_average_wkd FROM fb55.citibike
WHERE EXTRACT (DOW FROM starttime::date) IN (1,5)



'''
table = get_data(query)
table.head()

,trip_average_wkd,Unnamed: 1
0,678.375125,NaN


## Task 3 — Working with Space

In [25]:
# Showing the list of start station locations 
query = '''
SELECT start_station_id, start_station_longitude,
start_station_latitude
FROM fb55.citibike
GROUP BY start_station_id, start_station_longitude,
start_station_latitude
'''

table = get_data(query)
table.head()

,start_station_id,start_station_longitude,start_station_latitude
0,468,-73.981923,40.765265
1,164,-73.970325,40.753231
2,513,-73.988639,40.768254
3,430,-73.986569,40.701485
4,523,-73.991382,40.754666


In [16]:
# Showing the number of trips started per station
# … but only for stations within 500m of Time Square! 
# The coordinates of Time Square is (40.7577,-73.9857)
# Formulate referenced here but I used the radius in meter: http://daynebatten.com/2015/09/latitude-longitude-distance-sql/
query = '''SELECT start_station_id, COUNT(cartodb_id), 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2)) as Distance FROM fb55.citibike
WHERE 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2)) <= 500
GROUP BY start_station_id, start_station_latitude, start_station_longitude
'''
table = get_data(query)
table

,start_station_id,count,distance
0,2021,141,301.542817
1,524,112,344.084716
2,173,213,344.970448
3,529,221,445.636855
4,493,141,255.440158
5,465,251,294.801505
6,477,507,392.012009


In [17]:
#Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare! 
query = '''SELECT start_station_id, COUNT(cartodb_id), AVG(tripduration) as Mean_trip, 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2)) as Distance FROM fb55.citibike
WHERE 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2)) <= 500
AND 
EXTRACT (DOW FROM starttime::date) IN (6,0)
GROUP BY start_station_id, start_station_latitude, start_station_longitude
ORDER BY Mean_trip DESC;
'''
table = get_data(query)
table

,start_station_id,count,mean_trip,distance
0,173,48,1010.104167,344.970448
1,2021,44,762.931818,301.542817
2,465,33,683.121212,294.801505
3,493,15,675.400000,255.440158
4,477,73,643.260274,392.012009
5,524,10,629.700000,344.084716
6,529,28,579.142857,445.636855


## Extra:

In [29]:
# create lines for trips started: 
 #from stations within 500m of Times Squares
 #lasted less than 2 hours. 
 #The number of trips per each pair of stations are output as attributes of these lines. 
query = '''SELECT start_station_id, COUNT(cartodb_id), 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2)) as Distance_from_TimeSQ FROM fb55.citibike
WHERE 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2)) <= 500
AND 
tripduration/3600 <=2
GROUP BY start_station_id, 2 * 6374612 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(40.7577)) * cos(radians(start_station_latitude)) * (sin(radians((start_station_longitude - (-73.9857)) / 2))) ^ 2))
ORDER BY COUNT(cartodb_id) DESC;
'''
table = get_data(query)
table

,start_station_id,count,distance_from_timesq
0,477,507,392.012009
1,465,251,294.801505
2,529,221,445.636855
3,173,212,344.970448
4,493,141,255.440158
5,2021,141,301.542817
6,524,112,344.084716


# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel

**I added:**

import future

import sys
if sys.version_info.major > 2:
    xrange = range

import six
    

**I have tested on both Python 2 and 3 kernels.**

**This notebook seems to work compatibly.**